In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')

import PETsARD

In [2]:
loader = PETsARD.Loader(
    filepath='benchmark://adult',
    na_values={
        k: '?' for k in [
            'workclass',
            'occupation',
            'native-country'
        ]
    }
)
print(loader.data.head(1))


Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [3]:
from PETsARD.loader.metadata import Metadata

metadata = Metadata()
metadata.build_metadata(data=loader.data)
metadata.metadata

{'metadata_col': {'age': {'dtype': dtype('int8'),
   'na_percentage': 0.0,
   'infer_dtype': 'numerical'},
  'workclass': {'dtype': CategoricalDtype(categories=['Federal-gov', 'Local-gov', 'Never-worked', 'Private',
                     'Self-emp-inc', 'Self-emp-not-inc', 'State-gov',
                     'Without-pay'],
   , ordered=False),
   'na_percentage': 0.05730723557593874,
   'infer_dtype': 'categorical'},
  'fnlwgt': {'dtype': dtype('int32'),
   'na_percentage': 0.0,
   'infer_dtype': 'numerical'},
  'education': {'dtype': CategoricalDtype(categories=['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th',
                     '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate',
                     'HS-grad', 'Masters', 'Preschool', 'Prof-school',
                     'Some-college'],
   , ordered=False),
   'na_percentage': 0.0,
   'infer_dtype': 'categorical'},
  'educational-num': {'dtype': dtype('int8'),
   'na_percentage': 0.0,
   'infer_dtype': 'numerical'},
  

In [5]:
from pprint import pprint
pprint(loader.metadata)

{'metadata_col': {'age': {'dtype': dtype('int8'),
                          'infer_dtype': 'numerical',
                          'na_percentage': 0.0},
                  'capital-gain': {'dtype': dtype('int32'),
                                   'infer_dtype': 'numerical',
                                   'na_percentage': 0.0},
                  'capital-loss': {'dtype': dtype('int16'),
                                   'infer_dtype': 'numerical',
                                   'na_percentage': 0.0},
                  'education': {'dtype': CategoricalDtype(categories=['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th',
                  '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate',
                  'HS-grad', 'Masters', 'Preschool', 'Prof-school',
                  'Some-college'],
, ordered=False),
                                'infer_dtype': 'categorical',
                                'na_percentage': 0.0},
                  'educational-num': {'dty

In [3]:
import boto3
s3 = boto3.client('s3')
for bucket in s3.list_buckets()["Buckets"]:
    print(bucket["Name"])

cw-syn-results-319617166344-ap-northeast-1
pets-experiment
petsard-benchmark
petsard-benchmark-private
sagemaker-ap-northeast-1-319617166344
sagemaker-studio-319617166344-2h0qv3ot5q7
sagemaker-studio-319617166344-3plrsofl2jr


In [4]:
import hashlib

def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

print('benchmark/adult.csv')
print(calculate_sha256('benchmark/adult.csv'))
print('benchmark/adult_private_for_demo.csv')
print(calculate_sha256('benchmark/adult_private_for_demo.csv'))

benchmark/adult.csv
1f13ee2bf9d7c66098429281ab91fa1b51cbabd3b805cc365b3c6b44491ea2c0
benchmark/adult_private_for_demo.csv
1f13ee2bf9d7c66098429281ab91fa1b51cbabd3b805cc365b3c6b44491ea2c0


In [5]:
loader = PETsARD.Loader(
    filepath='benchmark://adult',
)
print(loader.data.head(1))

Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [6]:
loader = PETsARD.Loader(
    filepath='benchmark://adult-private',
)
print(loader.data.head(1))

Loader - Benchmarker: file benchmark\adult_private_for_demo.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
